<a href="https://colab.research.google.com/github/menotti/pp/blob/master/tutorials/Vetoriza%C3%A7%C3%A3o_SSE_%26_AVX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vetorização SSE & AVX

In [2]:
!git clone https://github.com/menotti/playground-41d1904q
!cat playground-41d1904q/techio.yml

fatal: destination path 'playground-41d1904q' already exists and is not an empty directory.
title : "Vetorização SSE & AVX "
cover: /images/cover.png
plan:
  - title: "O que é SSE e AVX?"
    statement: markdowns/intro.md
  - title: "Pré-requisitos"
    statement: markdowns/prereq.md
  - title: "Autovetorização"
    statement: markdowns/autovector.md
  - title: "Uso de SSE e AVX"
    statement: markdowns/usage.md
  - title: "Primeiro código AVX: cálculo SQRT"
    statement: markdowns/sqrt.md
  - title: "Frameworks C++ SSE/AVX"
    statement: markdowns/frameworks.md
  - title: "Mascaramento e carga condicional"
    statement: markdowns/masking.md
  - title: "Controlando o fluxo de dados"
    statement: markdowns/dataflow.md
  - title: "Palavras finais"
    statement: markdowns/final.md
projects:
    avx:
        root: /projects/avx
        buildCommand: "chmod +x prebuild.sh && ./prebuild.sh"
        runner:
            name: techio/cmake-gcc-runner
            version: 1.1.0-gcc-6.3
